1 - Teste diferentes regressores CV com a base house_prices
Aumente a complexidade da hipótese permitindo uma regressão polinomial, e deixe o CV decidir uma boa regularização!

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np

In [2]:
houses = pd.read_csv('house_prices.csv')

In [3]:
houses.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


In [4]:
houses.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,2919.000000,2919.000000,2433.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2896.000000,2918.000000,...,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,1460.000000
mean,1460.000000,57.137718,69.305795,10168.114080,6.089072,5.564577,1971.312778,1984.264474,102.201312,441.423235,...,93.709832,47.486811,23.098321,2.602261,16.062350,2.251799,50.825968,6.213087,2007.792737,180921.195890
std,842.787043,42.517628,23.344905,7886.996359,1.409947,1.113131,30.291442,20.894344,179.334253,455.610826,...,126.526589,67.575493,64.244246,25.188169,56.184365,35.663946,567.402211,2.714762,1.314964,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,730.500000,20.000000,59.000000,7478.000000,5.000000,5.000000,1953.500000,1965.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129975.000000
50%,1460.000000,50.000000,68.000000,9453.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,368.500000,...,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,2189.500000,70.000000,80.000000,11570.000000,7.000000,6.000000,2001.000000,2004.000000,164.000000,733.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,2919.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000,755000.000000


In [5]:
houses.shape

(2919, 81)

In [6]:
houses.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [7]:
#Checando as maiores correlaçoes com o preço de venda
houses.corr()['SalePrice'].nlargest(11).index

Index(['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea',
       'TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt',
       'YearRemodAdd'],
      dtype='object')

Devido a grande quantidade de colunas iremos descartar algumas que sejam decididas menos relevantes ou inadequadas

In [12]:
#separando X e y
colunas_relevantes = houses.corr()['SalePrice'].nlargest(11).index

houses_X = houses[colunas_relevantes].drop('SalePrice', axis=1)
houses_y = houses[['SalePrice']]

In [13]:
#Apenas 3 valores vazios em nossas features
houses_X.isna().sum().nlargest(10)

GarageCars      1
GarageArea      1
TotalBsmtSF     1
OverallQual     0
GrLivArea       0
1stFlrSF        0
FullBath        0
TotRmsAbvGrd    0
YearBuilt       0
YearRemodAdd    0
dtype: int64

In [10]:
#1459 valores vazios em nosso target
houses_y.isna().sum()

SalePrice    1459
dtype: int64

In [11]:
print(f'{round(100*1459/2919,2)}% dos dados de preço estão vazios')

49.98% dos dados de preço estão vazios


## Imputação dos dados faltantes
Vamos usar de uma imputação simples para os dados vazios, isso irá causar vazamento de dados quando os separarmos em treino e teste, por isso faremos cópias no momento para trabalhar.

In [31]:
X_train, X_test, y_train , y_test = train_test_split(houses_X, houses_y,test_size=0.3, random_state=42)

X_train = X_train.fillna(method="ffill")
X_test = X_test.fillna(method="ffill")
y_train = y_train.fillna(value=y_train.mean())
y_test = y_test.fillna(value=y_test.mean())

In [32]:
y_test.mean()

SalePrice    182285.371429
dtype: float64

In [33]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import ElasticNet
en = ElasticNet()
kf5 = KFold(n_splits=5, shuffle=True, random_state=42)
result_cv = cross_validate(
                            estimator=en, X=X_train, y=y_train, cv=kf5, 
                           scoring = "neg_mean_absolute_error", 
                           return_train_score=True
                           )

In [34]:
df_result_cv = pd.DataFrame(result_cv)
df_result_cv = df_result_cv.abs()
df_result_cv

,fit_time,score_time,test_score,train_score
0,0.064631,0.016610,31717.322985,29487.227553
1,0.010148,0.003016,28768.121523,30734.035598
2,0.013961,0.002992,29431.133255,30026.232523
3,0.012010,0.003018,27892.633804,30784.676883
4,0.013993,0.004006,33492.759208,29061.232607


In [35]:
df_result_cv[["test_score", "train_score"]].describe()

,test_score,train_score
count,5.000000,5.000000
mean,30260.394155,30018.681033
std,2296.283181,757.906510
min,27892.633804,29061.232607
25%,28768.121523,29487.227553
50%,29431.133255,30026.232523
75%,31717.322985,30734.035598
max,33492.759208,30784.676883
